In [1]:
import tensorflow as tf
import pdb


## 模型搭建

![](./image/self-attention.png)

## 构建Input组件

![](./image/input1.png)

下面市县各个功能的组件

### Norm Layer

In [2]:
def normalize(inputs, 
              epsilon = 1e-8,
              scope="ln",
              reuse=None):
    '''Applies layer normalization.

    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.

    Returns:
      A t
    '''
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)  # 计算 inputs的均值和方差
        beta = tf.Variable(tf.zeros(params_shape))
        gama = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ((variance + epsilon) ** (.5))
        
        outputs = gama *normalized + beta
        return outputs

### Embedding Layer

**embedding将大型稀疏向量转换为保留语义关系的低维空间。**embedding是一种矩阵，其中每列是与词汇表中的item对应的向量。 要获取单个词汇item的稠密向量，就检索与该item对应的列。

In [3]:
def embedding(inputs, 
              vocab_size, 
              num_units, 
              zero_pad=True, 
              scale=True,
              scope="embedding", 
              reuse=None):
    '''Embeds a given tensor.
    Args:
      inputs: A `Tensor` with type `int32` or `int64` containing the ids
         to be looked up in `lookup table`.
      vocab_size: An int. Vocabulary size.
      num_units: An int. Number of embedding hidden units.
      zero_pad: A boolean. If True, all the values of the fist row (id 0)
        should be constant zeros.
      scale: A boolean. If True. the outputs is multiplied by sqrt num_units.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
    Returns:
      A `Tensor` with one more rank than inputs's. The last dimensionality
        should be `num_units`.

    For example,

    ```
    import tensorflow as tf

    inputs = tf.to_int32(tf.reshape(tf.range(2*3), (2, 3)))
    outputs = embedding(inputs, 6, 2, zero_pad=True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print sess.run(outputs)
    >>
    [[[ 0.          0.        ]
      [ 0.09754146  0.67385566]
      [ 0.37864095 -0.35689294]]
     [[-1.01329422 -1.09939694]
      [ 0.7521342   0.38203377]
      [-0.04973143 -0.06210355]]]
    ```

    ```
    import tensorflow as tf

    inputs = tf.to_int32(tf.reshape(tf.range(2*3), (2, 3)))
    outputs = embedding(inputs, 6, 2, zero_pad=False)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print sess.run(outputs)
    >>
    [[[-0.19172323 -0.39159766]
      [-0.43212751 -0.66207761]
      [ 1.03452027 -0.26704335]]
     [[-0.11634696 -0.35983452]
      [ 0.50208133  0.53509563]
      [ 1.22204471 -0.96587461]]]
    ```
    '''
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable(name='lookup_table',
                                      dtype=tf.float32,
                                      shape=[vocab_size, num_units],
                                      initializer=tf.contrib.layers.xavier_initializer())
        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, inputs)    
        
        if scale:
                outputs = outputs * (num_units ** 0.5) 

    return outputs

In [4]:
# import tensorflow as tf
# import numpy as np

# inputs = tf.to_int32(np.array([[1]]))
# outputs = embedding(inputs, 50, 2, zero_pad=False)
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print(sess.run(outputs))

## Multihead Layer

![](./image/multihead1.png)

In [5]:
def multihead_attention(key_emb,
                        que_emb,
                        queries, 
                        keys, 
                        num_units=None, 
                        num_heads=8, 
                        dropout_rate=0,
                        is_training=True,
                        causality=False,
                        scope="multihead_attention", 
                        reuse=None):
    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      dropout_rate: A floating point number.
      is_training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=reuse):
#         pdb.set_trace()
        
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.layers.dense(queries, num_units, activation=tf.nn.relu) # (N, T_q, C)
        K = tf.layers.dense(keys, num_units, activation=tf.nn.relu) # (N, T_k, C)
        V = tf.layers.dense(keys, num_units, activation=tf.nn.relu) # (N, T_k, C)
        
        # Split and concat
        Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 

        # Multiplication
        outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)
        
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(key_emb, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
        
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
   
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
         
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(que_emb, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (N, T_q, C)
          
        # Dropouts
        outputs = tf.layers.dropout(outputs, rate=dropout_rate, training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs



## Feedforward

两层全连接，用卷积模拟加速运算，也可以使用dense层。

In [6]:
def feedforward(inputs, 
                num_units=[2048, 512],
                scope="multihead_attention", 
                reuse=None):
    '''Point-wise feed forward net.
    
    Args:
      inputs: A 3d tensor with shape of [N, T, C].
      num_units: A list of two integers.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns:
      A 3d tensor with the same shape and dtype as inputs
    '''
    
    with tf.variable_scope(scope, reuse=reuse):
        # Inner layer
        params = {"inputs": inputs, "filters": num_units[0], "kernel_size": 1,
                 "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Readout layer
        params = {"inputs": outputs, "filters": num_units[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}
        outputs = tf.layers.conv1d(**params)

        # Residual connection
        outputs += inputs
        
        # Normalize
        outputs = normalize(outputs)
    
    return outputs

## label_smoothing

对于训练有好处，将0变为接近零的小数，1变为接近1的数

In [7]:
def label_smoothing(inputs, epsilon=0.1):
    '''Applies label smoothing. See https://arxiv.org/abs/1512.00567.
    
    Args:
      inputs: A 3d tensor with shape of [N, T, V], where V is the number of vocabulary.
      epsilon: Smoothing rate.
    
    For example,
    
    ```
    import tensorflow as tf
    inputs = tf.convert_to_tensor([[[0, 0, 1], 
       [0, 1, 0],
       [1, 0, 0]],
      [[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]]], tf.float32)
       
    outputs = label_smoothing(inputs)
    
    with tf.Session() as sess:
        print(sess.run([outputs]))
    
    >>
    [array([[[ 0.03333334,  0.03333334,  0.93333334],
        [ 0.03333334,  0.93333334,  0.03333334],
        [ 0.93333334,  0.03333334,  0.03333334]],
       [[ 0.93333334,  0.03333334,  0.03333334],
        [ 0.93333334,  0.03333334,  0.03333334],
        [ 0.03333334,  0.93333334,  0.03333334]]], dtype=float32)]   
    ```
    '''
    K = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs) + (epsilon / K)


## 搭建模型

模型结构如下：
![](./image/model1.png)

In [8]:
import pdb

class TransformerModel():
    def __init__(self, is_training=True):
        tf.reset_default_graph()
        self.hidden_units = arg.hidden_units
        self.input_vocab_size = arg.input_vocab_size
        self.label_vocab_size = arg.label_vocab_size
        self.num_heads = arg.num_heads
        self.num_blocks = arg.num_blocks
        self.max_length = arg.max_length
        self.lr = arg.lr
        self.dropout = arg.dropout_rate
        
        # input
        self.x = tf.placeholder(tf.int32, shape=(None, None))
        self.y = tf.placeholder(tf.int32, shape=(None, None))
        self.de_inp = tf.placeholder(tf.int32, shape=(None, None))
        self.is_training = tf.placeholder(tf.bool)
        
        # Encoder
        with tf.variable_scope("encoder"):
            # embedding
            self.en_emb = embedding(self.x, vocab_size=self.input_vocab_size, num_units=self.hidden_units, scale=True, scope="enc_embed")
            self.enc = self.en_emb + embedding(tf.tile(tf.expand_dims(tf.range(tf.shape(self.x)[1]), 0), [tf.shape(self.x)[0], 1]),
                                          vocab_size=self.max_length,num_units=self.hidden_units, zero_pad=False, scale=False,scope="enc_pe")
#             pdb.set_trace()

            # Dropout
            self.enc = tf.layers.dropout(self.enc,rate=self.dropout, training=tf.convert_to_tensor(self.is_training))

            # Blocks
            for i in range(self.num_blocks):
                with tf.variable_scope("num_blocks-{}".format(i)):
                    # Multihead Attention
                    self.enc = multihead_attention(key_emb=self.en_emb,
                                                   que_emb=self.en_emb,
                                                  queries=self.enc,
                                                  keys=self.enc,
                                                  num_units=self.hidden_units,
                                                  dropout_rate=self.dropout,
                                                  is_training=self.is_training,
                                                  causality=False)

            # Feed Forward
            self.enc = feedforward(self.enc, num_units=[4*self.hidden_units, self.hidden_units])
        
        # Decoder
        with tf.variable_scope("decoder"):
            # embedding
            self.de_emb = embedding(self.de_inp, vocab_size=self.label_vocab_size, num_units=self.hidden_units, scale=True, scope="dec_embed")
            self.dec = self.de_emb + embedding(tf.tile(tf.expand_dims(tf.range(tf.shape(self.de_inp)[1]), 0), [tf.shape(self.de_inp)[0], 1]),
                                          vocab_size=self.max_length,num_units=self.hidden_units, zero_pad=False, scale=False,scope="dec_pe")

            ## Multihead Attention ( self-attention)
            for i in range(self.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i)):
                    ### Multihead Attention
                    self.dec = multihead_attention(key_emb = self.de_emb,
                                                   que_emb = self.de_emb,
                                                   queries=self.dec, 
                                                    keys=self.dec, 
                                                    num_units=self.hidden_units, 
                                                    num_heads=self.num_heads, 
                                                    dropout_rate=self.dropout,
                                                    is_training=self.is_training,
                                                    causality=True,
                                                    scope='self_attention')

            ## Multihead Attention ( vanilla attention)
            for i in range(self.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i)):
                    ### Multihead Attention
                    self.dec = multihead_attention(key_emb = self.en_emb,
                                                   que_emb = self.de_emb,
                                                   queries=self.dec, 
                                                    keys=self.enc, 
                                                    num_units=self.hidden_units, 
                                                    num_heads=self.num_heads, 
                                                    dropout_rate=self.dropout,
                                                    is_training=self.is_training,
                                                    causality=True,
                                                    scope='vanilla_attention') 

            ### Feed Forward
            self.outputs = feedforward(self.dec, num_units=[4*self.hidden_units, self.hidden_units])
        
        # 最终线性投影
        self.logits = tf.layers.dense(inputs=self.outputs, units=self.label_vocab_size, activation=None)
        self.preds = tf.to_int32(tf.arg_max(self.logits, -1))
        self.istarget = tf.to_float(tf.not_equal(self.y, 0))
        self.acc = tf.reduce_sum(tf.to_float(tf.equal(self.preds, self.y))*self.istarget) / (tf.reduce_sum(self.istarget))
        tf.summary.scalar('acc', self.acc)
        
        if is_training:
            #loss
            self.y_smoothed = label_smoothing(tf.one_hot(self.y, depth=self.label_vocab_size))
            self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y_smoothed)
            self.mean_loss = tf.reduce_sum(self.loss*self.istarget) / (tf.reduce_sum(self.istarget))
        
            # Training Scheme
            self.global_step = tf.Variable(0, name="global_step", trainable=False)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.9, beta2=0.98, epsilon=1e-8)
            self.train_op = self.optimizer.minimize(self.mean_loss, global_step=self.global_step)
            
            # Summary 
            tf.summary.scalar('mean_loss', self.mean_loss)
            self.merged = tf.summary.merge_all()

## 数据处理

读取数据，然后英文保存inputs，中文保存outputs

In [9]:
from tqdm import tqdm
import json

inputs = []
outputs = []
with open(file="H:\\PycharmProjects\\dataset\\translation2019zh\\translation2019zh_train.json", mode="r",encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        dic = json.loads(line)
        inputs.append(dic['english'].replace(',',' ,').lower())
        outputs.append(dic['chinese'])
        
train_data_num_start = 60000
train_data_num_end = train_data_num_start + 50000

inputs = inputs[train_data_num_start: train_data_num_end]
outputs = outputs[train_data_num_start: train_data_num_end]


100%|████████████████████████████████████████████████████████████████████| 5161434/5161434 [00:23<00:00, 222167.32it/s]


In [10]:
print(inputs[:10])

print(outputs[:10])

['the engine has been tuned with an ess vt2-600 supercharger setup , chosen for its copious power , race-proven performance , and reliability.', "great work! keep it up! don't stop working.", 'this means helping your child accept and learn from their mistakes or misbehaviour and supporting swis to ensure the good educational and personal progress of your child.', "but the cosmodrome is just a start for russia's new space ambitions.", 'be anchored in the bay where all men ride.', 'the bottom of lpg fractionator is separated into two spaces by baffle.', 'blindy had come up opposite us and we smelled him and turned around.', 'if we drill holes in strategic locations , we compromise the load-carrying capacity of the wall。', 'he removes the drill bit from his pocket and marks the floor. michael slides over to a yellow scaffolding beam while he lifts up his left sleeve.', 'make sure you try out different crafting stations , some may be better suited for particular recipes or trade skills tha

### 英文分词

英文只需要通过空格隔开就可以了

In [11]:
inputs = [en.split(' ') for en in tqdm(inputs) if en != ',']

inputs[:10]

100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 489750.78it/s]


[['the',
  'engine',
  'has',
  'been',
  'tuned',
  'with',
  'an',
  'ess',
  'vt2-600',
  'supercharger',
  'setup',
  ',',
  'chosen',
  'for',
  'its',
  'copious',
  'power',
  ',',
  'race-proven',
  'performance',
  ',',
  'and',
  'reliability.'],
 ['great', 'work!', 'keep', 'it', 'up!', "don't", 'stop', 'working.'],
 ['this',
  'means',
  'helping',
  'your',
  'child',
  'accept',
  'and',
  'learn',
  'from',
  'their',
  'mistakes',
  'or',
  'misbehaviour',
  'and',
  'supporting',
  'swis',
  'to',
  'ensure',
  'the',
  'good',
  'educational',
  'and',
  'personal',
  'progress',
  'of',
  'your',
  'child.'],
 ['but',
  'the',
  'cosmodrome',
  'is',
  'just',
  'a',
  'start',
  'for',
  "russia's",
  'new',
  'space',
  'ambitions.'],
 ['be', 'anchored', 'in', 'the', 'bay', 'where', 'all', 'men', 'ride.'],
 ['the',
  'bottom',
  'of',
  'lpg',
  'fractionator',
  'is',
  'separated',
  'into',
  'two',
  'spaces',
  'by',
  'baffle.'],
 ['blindy',
  'had',
  'come',

### 中文分词

通过结巴分词进行分词

In [12]:
import jieba

outputs = [[char for char in jieba.cut(line) if char != ' '] for line in tqdm(outputs)]

outputs[:10]

  0%|                                                                                        | 0/50000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\crixue\AppData\Local\Temp\jieba.cache
Loading model cost 0.730 seconds.
Prefix dict has been built succesfully.
100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:09<00:00, 5484.67it/s]


[['该',
  '引擎',
  '已经',
  '被',
  '调整',
  '与',
  '斯洛文尼亚',
  '就业',
  '服务局',
  'VT2',
  '-',
  '600',
  '增压器',
  '设置',
  '，',
  '其',
  '荧光棒',
  '权力',
  '，',
  '赛道',
  '上',
  '经过',
  '验证',
  '的',
  '性能',
  '选择',
  '，',
  '和',
  '可靠性',
  '。'],
 ['做', '得', '真棒', '！', '再接再厉', '哦', '！', '别', '停下', '前进', '的', '脚步', '。'],
 ['这',
  '意味着',
  '帮助',
  '孩子',
  '学会',
  '承认错误',
  '并',
  '从',
  '错误',
  '中',
  '学习',
  '；',
  '支持',
  '学校',
  '有助于',
  '孩子',
  '个人',
  '发展',
  '的',
  '良好',
  '教育',
  '。'],
 ['但', '航天器', '发射场', '只是', '俄罗斯', '新', '的', '空间', '雄心', '的', '开始', '。'],
 ['才', '在', '人', '尽', '可泊', '的', '港湾', '里', '抛锚', '。'],
 ['气体',
  '分馏',
  '装置',
  '的',
  '脱乙烷',
  '塔底',
  '设有',
  '隔板',
  '，',
  '将',
  '塔',
  '釜',
  '分为',
  '两个',
  '空间',
  '。'],
 ['这时',
  '盲汉',
  '已经',
  '走',
  '到',
  '我们',
  '对面',
  '，',
  '我们',
  '闻到',
  '他',
  '身上',
  '的',
  '气味',
  '，',
  '便',
  '转过身',
  '去',
  '。'],
 ['如果',
  '我们',
  '在',
  '关键',
  '位置',
  '钻孔',
  '，',
  '墙',
  '的',
  '承重',
  '强度',
  '就',
  '会',
  '降低',
  '。'],
 

##  生成字典

将英文和中文映射为id

In [13]:
def get_vocab_cache(fname='encoder_vocab_mapping.txt'):
    data = []
    if(os.path.exists(fname)):
        with open(fname, mode='r', encoding='utf-8') as r:
            data = r.read().split(' ')
    return data

def save_vocab_cache(data, fname='encoder_vocab_mapping.txt'):
    with open(fname, mode='w', encoding='utf-8') as w:
        for word in data:
            w.write(word + ' ')
        w.close()

In [ ]:
import numpy as np
import os

SOURCE_CODES = ['<PAD>']
TARGET_CODES = ['<PAD>', '<GO>', '<EOS>']

def get_vocab(data, init=['<PAD>'], cache_fname='encoder_vocab_mapping.txt'):
    vocab = init
    cache = get_vocab_cache(cache_fname)
    if(len(cache) > 0):
        vocab = cache
    
    for line in tqdm(data):
        for word in line:
            if word not in vocab:
                vocab.append(word)
    return vocab

encoder_vocab = get_vocab(inputs, init=SOURCE_CODES, cache_fname='encoder_vocab_mapping.txt')
decoder_vocab = get_vocab(outputs, init=TARGET_CODES, cache_fname='decoder_vocab_mapping.txt')

 20%|██████████████▉                                                             | 9820/50000 [00:24<01:46, 376.30it/s]

In [ ]:
print(encoder_vocab[:10])
print(decoder_vocab[-20:-10])

# save_vocab_cache(encoder_vocab, fname='encoder_vocab_mapping.txt')
# save_vocab_cache(decoder_vocab, fname='decoder_vocab_mapping.txt')
# get_vocab_cache(fname='encoder_vocab_mapping.txt')

##  数据生成器

翻译系统训练所需要的数据形式，跟谷歌gnmt输入致，gnmt的原理可以参考：https://github.com/tensorflow/nmt
大概是:

* 编码器输入：I am a student
* 解码器输入：(go) Je suis étudiant
* 解码器输出：Je suis étudiant (end)

即解码器输入起始部分有个开始符号，输出句尾有个结束符号。


In [ ]:
encoder_inputs = [[encoder_vocab.index(word) for word in line] for line in tqdm(inputs)]

In [ ]:
decoder_val_used = [[decoder_vocab.index(word) for word in line] for line in tqdm(outputs)]
decoder_inputs = [[decoder_vocab.index('<GO>')] + mapping_index for mapping_index in tqdm(decoder_val_used)]
decoder_targets = [mapping_index + [decoder_vocab.index('<EOS>')] for mapping_index in tqdm(decoder_val_used)]

In [ ]:
# decoder_inputs = [[decoder_vocab.index('<GO>')] + [decoder_vocab.index(word) for word in line] for line in tqdm(outputs)]
# decoder_targets = [[decoder_vocab.index(word) for word in line] + [decoder_vocab.index('<EOS>')] for line in tqdm(outputs)]

In [ ]:
print(decoder_inputs[:20])
print(decoder_targets[:20])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

def get_batch(encoder_inputs, decoder_inputs, decoder_targets, batch_size, batch_num):
    batch_num -= 1
    for k in (range(batch_num)):
        start = k * batch_size
        end = start + batch_size
        
        en_inputs_batch = encoder_inputs[start: end]
        de_inputs_batch = decoder_inputs[start: end]
        de_targets_batch = decoder_targets[start: end]
        
        en_len = [len(line) for line in en_inputs_batch]
        de_len = [len(line) for line in de_inputs_batch]
        
#         if len(en_len) == 0 or len(de_len) == 0:
#             continue
            
        max_words_len = 64
        max_en_len = max_words_len if max(en_len) > max_words_len else max(en_len)
        max_de_len = max_words_len if max(de_len) > max_words_len else max(de_len)
        
        en_inputs_batch = pad_sequences(sequences=en_inputs_batch, maxlen=max_en_len, padding='post', dtype='int32', value=0)
        de_inputs_batch = pad_sequences(sequences=de_inputs_batch, maxlen=max_de_len, padding='post', dtype='int32', value=0)
        de_targets_batch = pad_sequences(sequences=de_targets_batch, maxlen=max_de_len, padding='post', dtype='int32', value=0)
        yield en_inputs_batch, de_inputs_batch, de_targets_batch

        
batch_size = 4
batch = get_batch(encoder_inputs, decoder_inputs, decoder_targets, batch_size, len(encoder_inputs) // batch_size)
next(batch)


##  参数设定

In [ ]:
def create_hparams():
    params = tf.contrib.training.HParams(
        num_heads = 8,
        num_blocks = 6,
        # vocab
        input_vocab_size = 50,
        label_vocab_size = 50,
        # embedding size
        max_length = 100,
        hidden_units = 128,
        dropout_rate = 0.2,
        lr = 0.000005)
    return params

arg = create_hparams()
arg.input_vocab_size = len(encoder_vocab)
arg.label_vocab_size = len(decoder_vocab)

## 训练模型

In [ ]:
from sklearn.utils import shuffle
validate_ratio = 0.2

encoder_inputs, decoder_inputs, decoder_targets = shuffle(encoder_inputs, decoder_inputs, decoder_targets)
train_num = int(len(encoder_inputs) * (1 - validate_ratio))
train_encoder_inputs = encoder_inputs[:train_num]
val_encoder_inputs = encoder_inputs[train_num:]
train_decoder_inputs = decoder_inputs[:train_num]
val_decoder_inputs = decoder_inputs[train_num:]
train_decoder_targets = decoder_targets[:train_num]
val_decoder_targets = decoder_targets[train_num:]

print(train_encoder_inputs[:3])
print(train_decoder_inputs[:3])
print(train_decoder_targets[:3])

In [ ]:

import tensorflow as tf
import time

config = tf.ConfigProto()
config.gpu_options.allow_growth = True      #程序按需申请内存

epochs = 35
batch_size = 4

model = TransformerModel()

saver = tf.train.Saver(max_to_keep=5, keep_checkpoint_every_n_hours=1)
with tf.Session(config=config) as sess:
    merged = tf.summary.merge_all()
    sess.run(tf.global_variables_initializer())
    
    model_path = 'logs'
    if(os.path.exists(os.path.join(model_path, 'TransformerModel.meta'))):
        saver.restore(sess, "logs\TransformerModel")

    writer = tf.summary.FileWriter('tensorboard/train', tf.get_default_graph())
    validation_writer = tf.summary.FileWriter('tensorboard/validate', tf.get_default_graph())
    
    for k in range(epochs):
        total_loss = 0
        batch_num = len(train_encoder_inputs) // batch_size
        batch = get_batch(train_encoder_inputs, train_decoder_inputs, train_decoder_targets, batch_size, batch_num)
        val_batch_num = len(val_encoder_inputs) // batch_size
        val_batch = get_batch(val_encoder_inputs, val_decoder_inputs, val_decoder_targets, batch_size, val_batch_num)
        
        start_time = time.time()
        for i in range(batch_num):
            try:
                encoder_input, decoder_input, decoder_target = next(batch)
            except StopIteration:
                break
            feed = {model.x: encoder_input, model.y: decoder_input, model.de_inp: decoder_target, model.is_training: True}
            cost, _, acc = sess.run([model.mean_loss, model.train_op, model.acc], feed_dict=feed)
            total_loss += cost
            if (k * batch_num + i) % 10 == 0:
                val_encoder_input, val_decoder_input, val_decoder_target = next(val_batch)
                val_feed = {model.x: val_encoder_input, model.y: val_decoder_input, model.de_inp: val_decoder_target, model.is_training: False}
                val_acc = sess.run([model.acc], feed_dict=val_feed)
                
                print("batch num :", i,", train average loss:", total_loss/(i+1),", acc:", acc)
                print("validation current acc:", val_acc[0])
                
                rs = sess.run(merged, feed_dict=feed)
                writer.add_summary(rs, k*batch_num+i)
                
                vs = sess.run(merged, feed_dict=val_feed)
                validation_writer.add_summary(vs, k*batch_num+i)
        
        end_time = time.time()
        print('one epoch consume time:', (end_time-start_time), "s")
        print('=== epoch:', k+1, ' average loss: ', total_loss/(batch_num+1), " accuracy:", acc)
        print('-----**-----**-----**-----**-----**-----**-----**-----**-----**-----**-----')
    
    saver.save(sess=sess, save_path=os.path.join(model_path, 'TransformerModel'))
    writer.close()
    validation_writer.close()
            

## 模型测试和推理

### 先准备测试数据

In [ ]:
from tqdm import tqdm
import json
import jieba

test_inputs = []
test_outputs = []
with open(file="H:\\PycharmProjects\\dataset\\translation2019zh\\translation2019zh_valid.json", mode="r",encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        dic = json.loads(line)
        test_inputs.append(dic['english'].replace(',',' ,').lower())
        test_outputs.append(dic['chinese'])
        
test_data_ratio = 0.0125
test_inputs_ori, test_outputs_ori = shuffle(test_inputs, test_outputs)
test_data_num = int(len(test_inputs_ori) * test_data_ratio)
test_inputs = test_inputs_ori[:test_data_num]
test_outputs = test_outputs_ori[:test_data_num]
test_inputs = [en.split(' ') for en in tqdm(test_inputs) if en != ',']
test_outputs = [[char for char in jieba.cut(line) if char != ' '] for line in tqdm(test_outputs)]
test_encoder_vocab = get_vocab(test_inputs, init=SOURCE_CODES, cache_fname='encoder_vocab_mapping.txt')
test_decoder_vocab = get_vocab(test_outputs, init=TARGET_CODES, cache_fname='decoder_vocab_mapping.txt')
test_encoder_inputs = [[encoder_vocab.index(word) for word in line] for line in tqdm(test_inputs)]
test_decoder_val_used = [[test_decoder_vocab.index(word) for word in line] for line in tqdm(test_outputs)]
test_decoder_inputs = [[test_decoder_vocab.index('<GO>')] + mapping_index for mapping_index in tqdm(test_decoder_val_used)]
test_decoder_targets = [mapping_index + [test_decoder_vocab.index('<EOS>')] for mapping_index in tqdm(test_decoder_val_used)]



In [ ]:
def get_test_batch(encoder_inputs, decoder_inputs, decoder_targets):
    for i in range(len(encoder_inputs)):
        yield np.array([encoder_inputs[i]]), np.array([decoder_inputs[i]]), np.array([decoder_targets[i]])

test_batch = get_test_batch(test_encoder_inputs, test_decoder_inputs, test_decoder_targets)
next(test_batch)

In [ ]:
model = TransformerModel()

saver = tf.train.Saver()
model_path = 'logs'

with tf.Session() as sess:
    if(os.path.exists(os.path.join(model_path, 'TransformerModel.meta'))):
        saver.restore(sess, "logs\TransformerModel")
        
    test_acc_count = 0.0
    for j in range(len(test_encoder_inputs)):
        try:
            test_encoder_input, test_decoder_input, test_decoder_target = next(test_batch)
        except StopIteration:
            break
    
        de_inp = [[test_decoder_vocab.index('<GO>')]]
        test_feed = {model.x: test_encoder_input, model.de_inp: np.array(de_inp), model.is_training: False}
        preds = sess.run(model.preds, feed_dict=test_feed)
#         pdb.set_trace()
        if preds[0][-1] == test_decoder_vocab.index('<EOS>'):
            break
        de_inp[0].append(preds[0][-1]) 
        decoder_preds_words = ''.join([test_decoder_vocab[index] for index in de_inp[0][1:]])
        
        print("*****===*****===*****===*****===*****===*****===*****===*****")
        print("要求翻译的句子：", test_inputs_ori[j])
        print("预测翻译成：",decoder_preds_words)
        print("正确翻译为：", test_outputs_ori[j])
        print("*****===*****===*****===*****===*****===*****===*****===***** \n")
        decoder_preds_words = ''
        
